# SCORE2 Cardiovascular Risk Model

This notebook demonstrates the SCORE2 (Systematic COronary Risk Evaluation 2) cardiovascular risk prediction model.

## Model Overview

SCORE2 estimates 10-year risk of fatal and non-fatal cardiovascular disease based on:
- Age
- Sex
- Systolic blood pressure
- Total cholesterol
- HDL cholesterol
- Smoking status
- Regional CVD risk level

**Reference:** SCORE2 Working Group and ESC Cardiovascular Risk Collaboration. (2021). SCORE2 risk prediction algorithms: new models to estimate 10-year risk of cardiovascular disease in Europe. *European Heart Journal*, 42(25), 2439-2454.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from cvd_risk_calculator.models.score2 import SCORE2
from cvd_risk_calculator.core.validation import PatientData

# Set up plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)


## Example 1: Single Patient Calculation


In [ ]:
# Create a sample patient
patient = PatientData(
    age=55,
    sex="male",
    systolic_bp=140.0,
    total_cholesterol=6.0,
    hdl_cholesterol=1.2,
    smoking=True,
    region="moderate"
)

# Initialize model and calculate risk
model = SCORE2()
result = model.calculate(patient)

# Display results
print(f"Patient Profile:")
print(f"  Age: {patient.age} years")
print(f"  Sex: {patient.sex}")
print(f"  Systolic BP: {patient.systolic_bp} mmHg")
print(f"  Total Cholesterol: {patient.total_cholesterol} mmol/L")
print(f"  HDL Cholesterol: {patient.hdl_cholesterol} mmol/L")
print(f"  Smoking: {patient.smoking}")
print(f"  Region: {patient.region}")
print(f"\nSCORE2 Results:")
print(f"  10-year CVD risk: {result.risk_score:.1f}%")
print(f"  Risk category: {result.risk_category}")
print(f"  Model: {result.model_name} v{result.model_version}")


## Example 2: Batch Processing


In [ ]:
# Create a DataFrame with multiple patients
df = pd.DataFrame({
    'age': [55, 60, 45, 65, 50],
    'sex': ['male', 'female', 'male', 'male', 'female'],
    'systolic_bp': [140.0, 130.0, 150.0, 160.0, 125.0],
    'total_cholesterol': [6.0, 5.5, 7.0, 6.5, 5.0],
    'hdl_cholesterol': [1.2, 1.5, 1.0, 1.1, 1.6],
    'smoking': [True, False, True, True, False],
    'region': ['moderate', 'low', 'high', 'very_high', 'low']
})

# Calculate risks
results = model.calculate_batch(df)

# Display results
print("Batch Calculation Results:")
print(results[['age', 'sex', 'risk_score', 'risk_category']].to_string())

# Summary statistics
print(f"\nSummary:")
print(f"  Mean risk: {results['risk_score'].mean():.1f}%")
print(f"  Median risk: {results['risk_score'].median():.1f}%")
print(f"\nRisk Distribution:")
print(results['risk_category'].value_counts())


## Example 3: Risk Comparison Across Regions


In [ ]:
# Compare risk by region for the same patient profile
base_patient = {
    'age': 55,
    'sex': 'male',
    'systolic_bp': 140.0,
    'total_cholesterol': 6.0,
    'hdl_cholesterol': 1.2,
    'smoking': True
}

regions = ['low', 'moderate', 'high', 'very_high']
risks_by_region = {}

for region in regions:
    patient = PatientData(**base_patient, region=region)
    result = model.calculate(patient)
    risks_by_region[region] = result.risk_score

# Visualize
plt.figure(figsize=(8, 5))
plt.bar(risks_by_region.keys(), risks_by_region.values(), color='steelblue')
plt.xlabel('Region')
plt.ylabel('10-year CVD Risk (%)')
plt.title('SCORE2 Risk by Region (Same Patient Profile)')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## Example 4: Risk by Age and Sex


In [ ]:
# Calculate risk across different ages
ages = list(range(40, 70, 5))
risks_male = []
risks_female = []

for age in ages:
    # Male patient
    patient_m = PatientData(
        age=age, sex='male', systolic_bp=140.0,
        total_cholesterol=6.0, hdl_cholesterol=1.2,
        smoking=True, region='moderate'
    )
    result_m = model.calculate(patient_m)
    risks_male.append(result_m.risk_score)
    
    # Female patient
    patient_f = PatientData(
        age=age, sex='female', systolic_bp=140.0,
        total_cholesterol=6.0, hdl_cholesterol=1.2,
        smoking=True, region='moderate'
    )
    result_f = model.calculate(patient_f)
    risks_female.append(result_f.risk_score)

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(ages, risks_male, marker='o', label='Male', linewidth=2, markersize=8)
plt.plot(ages, risks_female, marker='s', label='Female', linewidth=2, markersize=8)
plt.xlabel('Age (years)', fontsize=12)
plt.ylabel('10-year CVD Risk (%)', fontsize=12)
plt.title('SCORE2 Risk by Age and Sex', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
